In [1]:
import os 

In [2]:
os.getcwd()

'd:\\mydlproject\\Kidney_DLproject\\research'

In [3]:
%pwd

'd:\\mydlproject\\Kidney_DLproject\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\mydlproject\\Kidney_DLproject'